In this Jupyter Notebook, we explore how to get data of properties and shared properties from Bucket. We use this input to draw line charts, bar charts and pie charts.

# Step 1 Connect to Bucket

We first start by loading the required packages.

In [ ]:
# Loading environment variables from the .env file
from dotenv import load_dotenv
import os
load_dotenv()
# Thing
from dcd.bucket.thing import Thing

We can connect to the Raspberry Pi Thing. We use the environment variable from the .env file. Note the parameter connect=False, avoiding the MQTT connection (not necessary for analysing data already collected).

In [ ]:
THING_ID = os.getenv("THING_ID", None)
PRIVATE_KEY_PATH = os.getenv("PRIVATE_KEY_PATH", None)
thing_pi = Thing(thing_id=THING_ID, private_key_path=PRIVATE_KEY_PATH, connect=False)
# show the details of the Thing, in JSON format
thing_pi.describe()

We can also connect to the Lightbulb Thing.

In [ ]:
LIGHTBULB_THING_ID = os.getenv("LIGHTBULB_THING_ID", None)
LIGHTBULB_PRIVATE_KEY_PATH = os.getenv("LIGHTBULB_PRIVATE_KEY_PATH", None)
thing_bulb = Thing(thing_id=LIGHTBULB_THING_ID, private_key_path=LIGHTBULB_PRIVATE_KEY_PATH, connect=False)
# show the details of the Thing, in JSON format
thing_bulb.describe()

# Step 2 Line charts and Properties

Here is an example that retrieve the light sensor property by its name and fetch its data between for September 28. Adjust the name of the property if you chose a different one (see on Bucket or in the above describe() output). To find a date to observe, look at the Bucket main dashboard and check when data has been collected.

In [ ]:
light = thing_pi.find_property_by_name('LDR sensor')
light.read(from_ts="2020-09-28 00:00:00", to_ts="2020-09-29 00:00:00")
print(len(light.values))

You can add light.describe() to have how the data look inside. You could also print len(light.values) to count the number of data points you received from Bucket. Once we fetched the data of our property, we can draw a line chart. For this we use Numpy, MatplotLib and Pandas, typical Python libraries for plotting and manipulating data.

In [ ]:
# ploting and math packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

The values as structured as follows: 

[
    [time, value 1, value 2],
    [time, value 1, value 2]
    ...
]

In the following snippet, we transform this array of values (a Python list) into a Numpy array which comes with handy ways of manipulating the data. For instance, we extract the first value of each array (the time) into a single array with data[:0], meaning for all rows, we take the column 0. Same for the value of the light, in the column 1. Then, we use Matplotlib to prepare a plot, which we provide with the time and the series (values of the light sensor).

In [ ]:
data = np.array(light.values)
time = data[:,0]
series = data[:, 1]
fig, axs = plt.subplots()
axs.plot(time, series)
plt.show()

The result could be nicer. First, we can format the time, transforming the UNIX timestamp into a Python date with Pandas. We can add label to each axis using the type information (in this case, in the first dimension). Finally, we add a grid in the background and we increase the width of the chart.

In [ ]:
data = np.array(light.values)
time = pd.to_datetime(data[:,0], unit='ms')
series = data[:, 1]

fig, axs = plt.subplots()
axs.plot(time, series)

axs.set_xlabel('time')
dimension = light.type["dimensions"][0]
axs.set_ylabel(dimension["name"] + " (" + dimension["unit"] + ")" )

axs.grid(True)
fig.set_figwidth(15)

plt.show()

The following snippets repeat a similar process with the CPU usage.

In [ ]:
cpu = thing_pi.find_property_by_name("Processor Usage")
cpu.read(from_ts="2020-09-28 06:00:00", to_ts="2020-09-28 09:00:00")

Here we illustrate several extra options such as setting a title and limiting the y axis from 0 to 40 (a fix scale makes plots more comparable).

In [ ]:
data = np.array(cpu.values)
time = pd.to_datetime(data[:,0], unit='ms')
series = data[:, 1].astype(float)

fig, axs = plt.subplots()
axs.plot(time, series)
axs.set_ylim(0, 40)
axs.set_title(cpu.name)
axs.set_xlabel('time')
dimension = cpu.type["dimensions"][0]
axs.set_ylabel(dimension["name"] + " (" + dimension["unit"] + ")" )
axs.grid(True)
fig.set_figwidth(15)

plt.show()

# Step 3 Muliple axis

The next step is about multiple dimensions such as the lightbulb status involving the ON/OFF state, the brightness and many other dimensions.

In [ ]:

lightbulb = thing_lightbulb.find_property_by_name("Lightbulb Status")
lightbulb.read(from_ts="2020-09-01 06:00:00", to_ts="2020-09-28 09:00:00")

Here is an example of chart with multiple axis. We use 'step' instead of 'plot' for the ON/OFF state, to get a staircase type of visualisation.

In [ ]:
data = np.array(bulb.values)
time = pd.to_datetime(data[:,0], unit='ms')
series1 = data[:, 1].astype(float)      # ON / OFF Status
series2 = data[:, 6].astype(float)      # Brightness

fig, axs = plt.subplots()
axs.step(time, series1, color="#00cc00")    # Any hexadecimal color would work
axs.set_ylim(0, 4)                          # The state is 0 or 1, but its visualisation is nicer with space above

# Create a second y-axis with twinx
axs2 = axs.twinx()
axs2.plot(time, series2, color="#cc0000")
axs2.set_ylim(0, 100)

axs.set_xlabel('time')

dim_state = lightbulb.type["dimensions"][0]
axs.set_ylabel(dim_state["name"])

dim_brigthness = lightbulb.type["dimensions"][5]
axs2.set_ylabel(dim_brigthness["name"])

axs.set_title(lightbulb.name)
axs.grid(True)
fig.set_figwidth(15)

plt.show()

# Step 4 Aggregation and Shared Properties

In the following snippet of code, we get the shared properties our Raspberry Pi Thing has access too.

In [ ]:
shared_properties = thing_pi.find_shared_properties(group = "*")
for property in thing_pi.shared_properties:
    property.describe()

With this list, we can for example loop through them and count the number of available data points.
Bucket does that for us, we can specify the following parameters:
* time_interval: the duration of each interval (1d for 1 day, 4h for 4 hours, 7m for 7 minute...)
* time_fct: the aggregation function for each interval: in this case we will 'count' the data points
* fill: what to do when there is no data, in our case we will fill with zeros.

In [ ]:
for property in shared_properties:
    prop_with_data = thing_pi.read_property(property.property_id, from_ts="2020-09-01 00:00:00", to_ts="2020-10-20 00:00:00", time_interval="1d", time_fct="count", fill="0")
    print(len(prop_with_data.values))

We can use this result to build a stacked bar chart, similar to the one on the Bucket dashboard.

In [ ]:

fig, axs = plt.subplots()
time = None
width = 0.20
previous = None
for property in shared_properties:
    prop_with_data = thing_pi.read_property(property.property_id, from_ts="2020-09-01 00:00:00", to_ts="2020-10-20 00:00:00", time_interval="1d", time_fct="count", fill="0")
    if len(prop_with_data.values) > 0:
        data = np.array(prop_with_data.values)
        # If we did not create x-axis (time) yet
        if time is None:
            time = pd.to_datetime(data[:,0], unit='ms')
        
        series = data[:, 1]     # count
        # Use the previous series as 'bottom' to stack bars of each properties
        axs.bar(time, series, width, bottom=previous, label=property.name)
        # The current series becomes the previous  
        previous = series                                   

axs.set_xlabel('time')
axs.set_ylabel("count")
axs.set_title('Count of all shared properties')
axs.legend()
fig.set_figwidth(15)
plt.show()


In this last example, we create a Pie chart using the count of . To automatically get a total count, we increase the time_interval to be at least as long as the whole time period we are looking at. In our case, 11w (for 11 week) covers roughly the period.

In [ ]:
labels = []
sizes = []
explode = []
for property in shared_properties:
    prop_with_data = thing_pi.read_property(property.property_id, from_ts="2020-09-01 00:00:00", to_ts="2020-10-20 00:00:00", time_interval="11w", time_fct="count")
    print(prop_with_data.values)
    if len(prop_with_data.values) > 0:
        labels.append(property.name)
        sizes.append(prop_with_data.values[0][1])
        # Let's give more importance to properties of type 'LIGHTBULB_STATUS'
        if property.type["id"] == "LIGHTBULB_STATUS":
            explode.append(0.1)
        else:
            explode.append(0)

# Build the Pie chart
fig, axs = plt.subplots()
# calculate pie chart with an explode section, labels,  without a shadow and starting at 90˚
axs.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=False, startangle=90)
axs.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
axs.legend()
plt.show()